### Creating and Using Pydantic Models

Pydantic Models are the foundations of data validation in Python. They use Python type validations to define the structure and validate data at runtime.

In [3]:
from pydantic import BaseModel

In [6]:
# create a Pydantic person class

class Person(BaseModel):
    name:str
    age:int
    city:str

person = Person(name='Emmanuel', age=40, city='Lubbock')
print(person)

name='Emmanuel' age=40 city='Lubbock'


* Attempting the above with a dataclass instead of Pydantic   

from dataclass import dataclass

@dataclass
class Person():
    name:str
    age:int
    city:str

person = Person(name='Emmanuel', age=40, city='Lubbock')

print(person)


* Note: dataclass does not ensure data validation like Pydantic

***Model with Optional Fields***

In [11]:
from typing import Optional
from pydantic import BaseModel, Field
from uuid import UUID, uuid4

In [15]:
class Employee(BaseModel):
    id:UUID = Field(default_factory=uuid4)  # generate a default uuid for me
    name:str
    department:str
    salary:Optional[float] = None          # None is Optional default value, but any float can be used to run the class
    is_active:Optional[bool] = True        # True is Optional default Value, but False can be used too to run the class


employee1 = Employee(name='John', department='IT')
print(employee1)

print('\n')

employee2 = Employee(name='Adam', department='HR', salary=6500.0, is_active=False)
print(employee2)

id=UUID('b0fa0501-08b0-4a5a-a56f-b0c1f58c6b9a') name='John' department='IT' salary=None is_active=True


id=UUID('2cf6ae3c-c84f-40bf-b5ba-e3d6bd400636') name='Adam' department='HR' salary=6500.0 is_active=False


***Model with List Field***

In [16]:
from pydantic import BaseModel
from typing import List

In [17]:
class Classroom(BaseModel):
    room_number:str
    students:List[str]       # List of Strings
    capacity:int

classroom = Classroom(
    room_number='A101',
    students=['Alice', 'Tonia', 'Jake'],      # tuple() can also be used instead of list []. The data will be typecasted to List when run
    capacity=30
)

print(classroom)

room_number='A101' students=['Alice', 'Tonia', 'Jake'] capacity=30


***Using Validation Error or Value Error***

In [18]:
from pydantic import BaseModel, ValidationError

try:
    invalid_user = Classroom(
        room_number = 'A1',
        students = ('Emma', 123),
        capacity = 100
    )
except ValidationError as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type


***Nested Model***

In [24]:
class Address(BaseModel):
    street:str
    city:str
    zip_code:int

class Customer(BaseModel):
    customer_id:UUID = Field(default_factory=uuid4)
    name:str
    address:Address       # Nested Model


customer = Customer(
    name = 'Emmanuel',
    address = {
        'street': '2721 41st Street',
        'city': 'Lubbock',
        'zip_code': 79413                   # '79413' also works (it will be typecasted as int)
    }
)

print(customer)

customer_id=UUID('d3aa0bda-02dd-47fd-ba7d-3201f214788c') name='Emmanuel' address=Address(street='2721 41st Street', city='Lubbock', zip_code=79413)


***Customization and Constraint***

In [26]:
from pydantic import BaseModel, Field

class Items(BaseModel):
    name:str = Field(min_length=2, max_length=50)
    price:float = Field(gt=0, le=1000)         # > 0 and <= 1000)
    quantity:int = Field(ge=0)

item1 = Items(
    name = 'Book',
    price = 30.5,
    quantity = 1000
)

print(item1)

name='Book' price=30.5 quantity=1000


In [27]:
item2 = Items(
    name = 'Apples',
    price = 10000,
    quantity = 1000
)

print(item2)                     # I want to get an Validation error here

ValidationError: 1 validation error for Items
price
  Input should be less than or equal to 1000 [type=less_than_equal, input_value=10000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

***Using Description with Fields***

In [28]:
class User(BaseModel):
    username: str = Field(..., description = 'Unique username for the user')
    age: int = Field(default=18, description = 'User age, defaults to 18')
    email: str = Field(default_factory=lambda: 'user@example.com', description = 'default email address')

user1 = User(username = 'Alice')
print(user1, '\n')

user2 = User(username='Bob', age=20, email='bob@example.com')
print(user2)

username='Alice' age=18 email='user@example.com' 

username='Bob' age=20 email='bob@example.com'


In [29]:
# Get the schema for the Class User

print(User.schema())

{'properties': {'username': {'description': 'Unique username for the user', 'title': 'Username', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age, defaults to 18', 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'default email address', 'title': 'Email', 'type': 'string'}}, 'required': ['username'], 'title': 'User', 'type': 'object'}


/tmp/ipykernel_3996/275923770.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(User.schema())


In [30]:
print(User.model_json_schema())

{'properties': {'username': {'description': 'Unique username for the user', 'title': 'Username', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age, defaults to 18', 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'default email address', 'title': 'Email', 'type': 'string'}}, 'required': ['username'], 'title': 'User', 'type': 'object'}
